<a href="https://colab.research.google.com/github/mpratiek/END_NLP/blob/main/END3_Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
# !wget https://download.pytorch.org/tutorial/data.zip

!unzip data.zip

--2021-11-25 16:03:58--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv.1’

quora_duplicate_que 100%[===================>]  55.48M   145MB/s    in 0.4s    

2021-11-25 16:03:58 (145 MB/s) - ‘quora_duplicate_questions.tsv.1’ saved [58176133/58176133]

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [148]:
df = pd.read_csv("/content/quora_duplicate_questions.tsv", sep='\t')

In [149]:
df = df[df['is_duplicate'] ==1]

In [150]:
df

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
...,...,...,...,...,...,...
404280,404280,537922,537923,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,1
404281,404281,99131,81495,Why is Manaphy childish in Pokémon Ranger and ...,Why is Manaphy annoying in Pokemon ranger and ...,1
404282,404282,1931,16773,How does a long distance relationship work?,How are long distance relationships maintained?,1
404284,404284,537926,537927,What does Jainism say about homosexuality?,What does Jainism say about Gays and Homosexua...,1


In [151]:
df["question2"].str.len().max()

295

In [152]:
df['input'] = df["question1"] + '\t' + df["question2"]

In [153]:
df['input'][5].split("\t")

['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"]

In [154]:
li = df['input'].to_list()

In [155]:
with open('eng-eng.txt', 'w') as f:
  for i in li:
    f.write(i)
    f.write('\n')

In [156]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 295

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 149263 sentence pairs
Trimmed to 679 sentence pairs
Counting words...
Counted words:
eng 1506
eng 1475
['what are the best was to lose weight ?', 'i m fat . how do i lose weight ?']


In [93]:
# input_lang, output_lang, pairs = readLangs('eng', 'fra', reverse=False)

In [94]:
# lines = open('/content/%s-%s.txt' % ('eng', 'eng'), encoding='utf-8').\
#         read().strip().split('\n')

#     # Split every line into pairs and normalize
# pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

In [95]:
# for l in lines:
#   print(l.split("\t"))
#   break

In [96]:
type(pairs)

list

In [157]:
pairs[0:5]

[['i am my weight is kg and my height is . ft . what should i do to gain height and weight ?',
  'i m year old my height is feet inches and my weight is kg what should i do to gain weight ?'],
 ['what can i do when i m ?', 'i m . what should i do ?'],
 ['how is trump becoming the president affect the indians applying for an ms in the us mech ?',
  'i am an indian planning to go to us for ms a stem course this january . if trump wins how will that affect my future in us ?'],
 ['kindly tell me whole process of admission at vits vellore for biotech .i m a bio student in . .i don t have math there ?',
  'i m bio student how i got admission in vits vellore in biotechnology and i don t have math in ?'],
 ['what can i do to improve my english grammar ?',
  'i am poor in english grammar so how should i improve my grammar ?']]

# The architecture we are building

![image](https://miro.medium.com/max/1838/1*tXchCn0hBSUau3WO0ViD7w.jpeg)

As we can see here, we will have an encoder, an attention mechanism block and decoder. In the final code the attention mechanicm block and decoder will be merged into single block as we need both to work together. 

As we can see here, we need to create a copy of h1, h2, h3 and h4. These are encoder outputs for a sentence with 4 words. 

# Encoder

We will build our encoder with a GRU, but that's all we know. Let's NOT strait away build a class, but see how to come up with one for the Encoder. We need to answer few questions first:
1. what would be the hidden size of our GRU
2. What would be the input size
3. What would be the embedding dimesions. 

For simplicity, lets keep 1. and 3. to be 256. 

We can't feed our input directly to GRU, we need to tensorize it, convert to embeddings first. 

`embedding = nn.Embedding(input_size, hidden_size) `

## What is input_size?

Remember the line below?

`input_lang, output_lang, pairs = prepareData('eng', 'fra', True)`

In [98]:
input_lang

In [99]:
help(input_lang)

Help on Lang in module __main__ object:

class Lang(builtins.object)
 |  Lang(name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addSentence(self, sentence)
 |  
 |  addWord(self, word)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [100]:
input_lang.__dict__.items()

dict_items([('name', 'eng'), ('word2index', {'i': 2, 'am': 3, 'my': 4, 'weight': 5, 'is': 6, 'kg': 7, 'and': 8, 'height': 9, '.': 10, 'ft': 11, 'what': 12, 'should': 13, 'do': 14, 'to': 15, 'gain': 16, '?': 17, 'can': 18, 'when': 19, 'm': 20, 'how': 21, 'trump': 22, 'becoming': 23, 'the': 24, 'president': 25, 'affect': 26, 'indians': 27, 'applying': 28, 'for': 29, 'an': 30, 'ms': 31, 'in': 32, 'us': 33, 'mech': 34, 'kindly': 35, 'tell': 36, 'me': 37, 'whole': 38, 'process': 39, 'of': 40, 'admission': 41, 'at': 42, 'vits': 43, 'vellore': 44, 'biotech': 45, '.i': 46, 'a': 47, 'bio': 48, 'student': 49, 'don': 50, 't': 51, 'have': 52, 'math': 53, 'there': 54, 'improve': 55, 'english': 56, 'grammar': 57, 'easiest': 58, 'way': 59, 'earn': 60, 'money': 61, 'from': 62, 'online': 63, 'good': 64, 'shape': 65, 'but': 66, 'trouble': 67, 'spot': 68, 'cellulite': 69, 'right': 70, 'under': 71, 'glutes': 72, 'top': 73, 'thighs': 74, 'get': 75, 'rid': 76, 'this': 77, 'petite': 78, 'year': 79, 'old': 80

In [101]:
input_size = input_lang.n_words
hidden_size = 256
input_size

1506

In [102]:
embedding = nn.Embedding(input_size, hidden_size)
gru = nn.GRU(hidden_size, hidden_size)

In [103]:
sample = random.choice(pairs)
sample

['what programming languages should i learn for video game development ?',
 'i am a year old boy that wants to learn how to program video games . what programming languages should i learn ? how do i get started ?']

In [104]:
device

device(type='cpu')

In [105]:
embedding_input = embedding(sample[0])

TypeError: ignored

In [ ]:
sample

In [ ]:
input_sentence = sample[0]
output_sentence = sample[1]

input_lang.word2index['money']

In [ ]:
for word in input_sentence:
  print(word)

In [ ]:
for word in input_sentence.split(' '):
  print(word)

In [ ]:
input_indices = [input_lang.word2index[word] for word in input_sentence.split(' ')]
output_indices = [output_lang.word2index[word] for word in output_sentence.split(' ')]
input_indices, output_indices

In [ ]:
embedding_input = embedding(input_indices)

In [ ]:
input_indices.append(EOS_token)
output_indices.append(EOS_token)
input_indices, output_indices

In [ ]:
input_tensor = torch.tensor(input_indices, dtype=torch.long, device=device)
output_tensor = torch.tensor(output_indices, dtype=torch.long, device=device)

In [ ]:
input_tensor.shape, output_tensor.shape

In [ ]:
embedding_input = embedding(input_tensor)

In [ ]:
embedding = nn.Embedding(input_size, hidden_size).to(device)
gru = nn.GRU(hidden_size, hidden_size).to(device)

In [ ]:
embedding_input = embedding(input_tensor)
embedding_input.shape

In [ ]:
input_tensor

In [ ]:
input_tensor.shape, input_tensor.view(-1, 1).shape

In [ ]:
print(embedding_input.shape)
embedding_input = embedding(input_tensor.view(-1, 1))
print(embedding_input.shape)

In [ ]:
# output, hidden = gru(embedde_input, ?)
hidden = torch.zeros(1, 1, 256, device = device)

In [ ]:
embedding_input = embedding(input_tensor.view(-1, 1))
output, hidden = gru(embedding_input, hidden)

output.shape, output[0, 0].shape

In [ ]:
encoder_outputs = torch.zeros(MAX_LENGTH, 256, device=device)
encoder_outputs.shape

In [ ]:
input_tensor.size()[0]

In [ ]:
encoder_outputs = torch.zeros(MAX_LENGTH, 256, device=device)
hidden = torch.zeros(1, 1, 256, device = device)

for i in range(input_tensor.size()[0]):
  embedding_input = embedding(input_tensor[i].view(-1, 1))
  output, hidden = gru(embedding_input, hidden)
  encoder_outputs[i] += output[0, 0]

In [ ]:
encoder_outputs.shape, hidden.shape

In [ ]:
encoder_outputs[0:4]

In [ ]:
encoder_outputs[7:10]

# 😁

Finally our Encoder is fully ready. Now let's look at the class we wrote in the last class to see what we missed!

```
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
```

Cool! Next let's build out Decoder where we have attention in-built.

# Decoder with Attention

Here is the plan. 

1. First input to the decoder will be SOS_token, later inputs would be the words it predicted (unless we implement teacher forcing)
2. decoder/GRU's hidden state will be initialized with the encoder's last hidden state
3. we will use gru's hidden state and last prediction to generate attention weight using a FC layer. 
4. this attention weight will be used to weigh the encoder_outputs using batch matric multiplication. This will give us a NEW view on how to look at encoder_states.
5. this attention applied encoder_states will then be concatenated with the input, and then sent a linear layer and _then_ sent to the GRU. 
6. GRU's output will be sent to a FC layer to predict one of the output_language words

Let's prepare all the inputs we need to do this


In [ ]:
decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = hidden
decoded_words = []

In [ ]:
# decoder s0
output_size = output_lang.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
embedded.shape

In [ ]:
# 256 * 2 >> after concatenation
attn_weight_layer = nn.Linear(256 * 2, 295).to(device)

In [ ]:
embedded.shape, decoder_hidden.shape

In [106]:
torch.cat((embedded[0], decoder_hidden[0]), 1).shape

torch.Size([1, 512])

In [107]:
attn_weight_layer = nn.Linear(256 * 2, 295).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights

tensor([[-0.0696,  0.5946, -0.2599,  0.0091, -0.2947, -0.3255, -0.3279, -0.0034,
          0.3906,  0.1648, -0.3008,  0.7182,  0.7062,  0.0771,  0.4085,  0.7563,
         -0.4063, -0.5761,  0.4507, -0.1044,  0.0579,  0.0293, -0.4032, -0.3035,
          0.3577,  0.4141,  0.0284, -0.7234, -0.8046, -0.9319, -0.3895, -0.0679,
          0.6483, -1.0833, -0.1429,  0.0319, -0.1617,  0.4080,  0.0823, -0.3470,
         -0.3136,  0.3890, -0.3417, -1.1319,  0.2788, -0.5906, -1.4668,  0.1593,
         -0.3987,  0.4750, -0.0640,  0.3848, -0.3006, -0.1680,  0.4830,  0.1867,
         -0.1346, -0.8271,  0.4298, -0.1266,  0.0286,  0.1157,  0.5742,  0.4863,
          0.4063,  0.0867, -0.0673,  0.6247, -0.3713, -0.5740,  0.3602,  0.3679,
         -0.5224, -0.5025,  0.4599,  0.2808, -0.3272,  0.5404,  0.3917, -0.1459,
         -0.3649,  0.5241, -0.0749, -0.4869,  0.0972, -0.1045, -0.7785,  0.2313,
          0.8983,  0.3396,  0.2445, -0.1928,  0.2112, -0.2934,  0.7380, -0.4011,
          0.2409,  0.0931, -

In [108]:
import torch.nn.functional as F
attn_weight_layer = nn.Linear(256 * 2, 295).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_weights


tensor([[0.0032, 0.0032, 0.0007, 0.0044, 0.0016, 0.0027, 0.0038, 0.0047, 0.0047,
         0.0016, 0.0072, 0.0034, 0.0023, 0.0016, 0.0030, 0.0027, 0.0038, 0.0100,
         0.0040, 0.0014, 0.0015, 0.0021, 0.0030, 0.0048, 0.0029, 0.0015, 0.0030,
         0.0040, 0.0014, 0.0014, 0.0030, 0.0032, 0.0044, 0.0023, 0.0039, 0.0026,
         0.0016, 0.0015, 0.0034, 0.0107, 0.0047, 0.0017, 0.0039, 0.0025, 0.0058,
         0.0015, 0.0035, 0.0025, 0.0024, 0.0033, 0.0051, 0.0079, 0.0025, 0.0034,
         0.0018, 0.0018, 0.0014, 0.0031, 0.0035, 0.0020, 0.0024, 0.0036, 0.0050,
         0.0041, 0.0033, 0.0041, 0.0048, 0.0036, 0.0044, 0.0029, 0.0037, 0.0021,
         0.0037, 0.0030, 0.0073, 0.0056, 0.0063, 0.0040, 0.0026, 0.0024, 0.0028,
         0.0046, 0.0031, 0.0017, 0.0027, 0.0031, 0.0029, 0.0067, 0.0031, 0.0052,
         0.0035, 0.0023, 0.0023, 0.0027, 0.0020, 0.0020, 0.0046, 0.0016, 0.0030,
         0.0018, 0.0016, 0.0018, 0.0014, 0.0020, 0.0032, 0.0039, 0.0042, 0.0020,
         0.0049, 0.0052, 0.0

In [109]:
attn_weights.shape, encoder_outputs.shape

(torch.Size([1, 295]), torch.Size([295, 256]))

In [110]:
attn_applied = torch.bmm(attn_weights, encoder_outputs)

RuntimeError: ignored

In [ ]:
attn_weights.unsqueeze(0).shape, encoder_outputs.unsqueeze(0).shape

In [111]:
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
attn_applied.shape

torch.Size([1, 1, 256])

So, now we have this 256dm attn_applied encoder_outputs capturing what we should focus on on this step. We also have the input we already generated. That's 256dm again. GRU is gonna take 256 only. So we need to concatenate them, send to a linear layer to reduce dimensions, and then send to Gru
![image](https://static.wikia.nocookie.net/mycun-the-movie/images/c/c2/Gru-icon.png/revision/latest/scale-to-width-down/250?cb=20151223171656)

In [112]:
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
embedded.shape, attn_applied.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [113]:
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru.shape

torch.Size([1, 256])

In [114]:
gru = nn.GRU(256, 256).to(device)
decoder_hidden.shape, input_to_gru.shape

(torch.Size([1, 1, 256]), torch.Size([1, 256]))

In [115]:
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
decoder_hidden.shape, input_to_gru.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [116]:
output, decoder_hidden = gru(decoder_hidden, input_to_gru)
output.shape, decoder_hidden.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [117]:
output_word_layer = nn.Linear(256, output_lang.n_words).to(device)

In [118]:
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim=1)
output.shape, output, output.data.topk(1)

(torch.Size([1, 1475]),
 tensor([[0.0007, 0.0007, 0.0008,  ..., 0.0006, 0.0008, 0.0006]],
        grad_fn=<SoftmaxBackward0>),
 torch.return_types.topk(values=tensor([[0.0009]]), indices=tensor([[637]])))

In [119]:
topv, topi = output.data.topk(1)
output_lang.index2word[topi.item()]

'little'

In [120]:
decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = hidden #decoder_hidden = encoder_hidden
output_size = output_lang.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
attn_weight_layer = nn.Linear(256 * 2, 295).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
gru = nn.GRU(256, 256).to(device)
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output_word_layer = nn.Linear(256, output_lang.n_words).to(device)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_lang.index2word[top_index.item()]



'brand'

In [121]:
embedding = nn.Embedding(output_size, 256).to(device)
attn_weight_layer = nn.Linear(256 * 2, 295).to(device)
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
gru = nn.GRU(256, 256).to(device)
output_word_layer = nn.Linear(256, output_lang.n_words).to(device)

decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = hidden
output_size = output_lang.n_words
embedded = embedding(decoder_input)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_lang.index2word[top_index.item()], attn_weights

('french',
 tensor([[0.0045, 0.0033, 0.0013, 0.0028, 0.0028, 0.0024, 0.0020, 0.0017, 0.0081,
          0.0035, 0.0025, 0.0038, 0.0067, 0.0029, 0.0043, 0.0037, 0.0083, 0.0044,
          0.0034, 0.0015, 0.0027, 0.0019, 0.0026, 0.0032, 0.0034, 0.0027, 0.0038,
          0.0016, 0.0027, 0.0056, 0.0052, 0.0031, 0.0023, 0.0039, 0.0029, 0.0048,
          0.0042, 0.0033, 0.0020, 0.0044, 0.0031, 0.0053, 0.0013, 0.0033, 0.0055,
          0.0048, 0.0022, 0.0027, 0.0016, 0.0044, 0.0015, 0.0027, 0.0032, 0.0078,
          0.0056, 0.0012, 0.0052, 0.0041, 0.0036, 0.0028, 0.0029, 0.0049, 0.0028,
          0.0046, 0.0043, 0.0040, 0.0031, 0.0033, 0.0043, 0.0031, 0.0054, 0.0021,
          0.0024, 0.0015, 0.0010, 0.0050, 0.0019, 0.0016, 0.0055, 0.0017, 0.0017,
          0.0023, 0.0025, 0.0024, 0.0024, 0.0021, 0.0035, 0.0025, 0.0059, 0.0062,
          0.0047, 0.0008, 0.0015, 0.0028, 0.0014, 0.0023, 0.0051, 0.0021, 0.0027,
          0.0015, 0.0022, 0.0021, 0.0045, 0.0028, 0.0028, 0.0039, 0.0027, 0.0024,
     

In [122]:
decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = hidden
output_size = output_lang.n_words
embedded = embedding(decoder_input)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_lang.index2word[top_index.item()], attn_weights

('french',
 tensor([[0.0045, 0.0033, 0.0013, 0.0028, 0.0028, 0.0024, 0.0020, 0.0017, 0.0081,
          0.0035, 0.0025, 0.0038, 0.0067, 0.0029, 0.0043, 0.0037, 0.0083, 0.0044,
          0.0034, 0.0015, 0.0027, 0.0019, 0.0026, 0.0032, 0.0034, 0.0027, 0.0038,
          0.0016, 0.0027, 0.0056, 0.0052, 0.0031, 0.0023, 0.0039, 0.0029, 0.0048,
          0.0042, 0.0033, 0.0020, 0.0044, 0.0031, 0.0053, 0.0013, 0.0033, 0.0055,
          0.0048, 0.0022, 0.0027, 0.0016, 0.0044, 0.0015, 0.0027, 0.0032, 0.0078,
          0.0056, 0.0012, 0.0052, 0.0041, 0.0036, 0.0028, 0.0029, 0.0049, 0.0028,
          0.0046, 0.0043, 0.0040, 0.0031, 0.0033, 0.0043, 0.0031, 0.0054, 0.0021,
          0.0024, 0.0015, 0.0010, 0.0050, 0.0019, 0.0016, 0.0055, 0.0017, 0.0017,
          0.0023, 0.0025, 0.0024, 0.0024, 0.0021, 0.0035, 0.0025, 0.0059, 0.0062,
          0.0047, 0.0008, 0.0015, 0.0028, 0.0014, 0.0023, 0.0051, 0.0021, 0.0027,
          0.0015, 0.0022, 0.0021, 0.0045, 0.0028, 0.0028, 0.0039, 0.0027, 0.0024,
     

In [123]:
decoder_input = torch.tensor([[top_index.item()]], device=device)
decoder_hidden = hidden
output_size = output_lang.n_words
embedded = embedding(decoder_input)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_lang.index2word[top_index.item()], attn_weights

('morning',
 tensor([[0.0028, 0.0074, 0.0022, 0.0016, 0.0017, 0.0055, 0.0044, 0.0017, 0.0058,
          0.0020, 0.0023, 0.0034, 0.0029, 0.0043, 0.0024, 0.0025, 0.0029, 0.0029,
          0.0046, 0.0039, 0.0010, 0.0029, 0.0021, 0.0026, 0.0036, 0.0033, 0.0023,
          0.0033, 0.0044, 0.0028, 0.0023, 0.0038, 0.0021, 0.0043, 0.0048, 0.0046,
          0.0054, 0.0029, 0.0020, 0.0021, 0.0031, 0.0025, 0.0013, 0.0067, 0.0016,
          0.0056, 0.0024, 0.0061, 0.0033, 0.0024, 0.0038, 0.0037, 0.0043, 0.0037,
          0.0036, 0.0045, 0.0027, 0.0037, 0.0040, 0.0046, 0.0044, 0.0018, 0.0041,
          0.0028, 0.0066, 0.0032, 0.0021, 0.0028, 0.0026, 0.0022, 0.0041, 0.0013,
          0.0063, 0.0012, 0.0015, 0.0026, 0.0028, 0.0045, 0.0021, 0.0031, 0.0049,
          0.0021, 0.0019, 0.0046, 0.0059, 0.0057, 0.0044, 0.0051, 0.0046, 0.0023,
          0.0018, 0.0018, 0.0062, 0.0023, 0.0048, 0.0061, 0.0010, 0.0049, 0.0044,
          0.0040, 0.0029, 0.0038, 0.0035, 0.0023, 0.0016, 0.0038, 0.0024, 0.0039,
    

In [124]:
decoder_input = torch.tensor([[top_index.item()]], device=device)
decoder_hidden = hidden
output_size = output_lang.n_words
embedded = embedding(decoder_input)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_lang.index2word[top_index.item()], attn_weights

('same',
 tensor([[0.0046, 0.0020, 0.0036, 0.0025, 0.0033, 0.0024, 0.0030, 0.0041, 0.0030,
          0.0024, 0.0013, 0.0053, 0.0046, 0.0031, 0.0043, 0.0025, 0.0023, 0.0035,
          0.0041, 0.0027, 0.0047, 0.0026, 0.0075, 0.0036, 0.0023, 0.0019, 0.0024,
          0.0029, 0.0024, 0.0054, 0.0050, 0.0039, 0.0024, 0.0021, 0.0034, 0.0021,
          0.0052, 0.0029, 0.0037, 0.0031, 0.0050, 0.0027, 0.0030, 0.0030, 0.0040,
          0.0037, 0.0014, 0.0034, 0.0016, 0.0023, 0.0034, 0.0017, 0.0028, 0.0049,
          0.0045, 0.0026, 0.0038, 0.0025, 0.0050, 0.0052, 0.0009, 0.0014, 0.0034,
          0.0049, 0.0049, 0.0015, 0.0013, 0.0036, 0.0022, 0.0037, 0.0013, 0.0035,
          0.0040, 0.0014, 0.0012, 0.0048, 0.0016, 0.0022, 0.0026, 0.0015, 0.0029,
          0.0054, 0.0020, 0.0036, 0.0045, 0.0042, 0.0035, 0.0031, 0.0018, 0.0060,
          0.0030, 0.0029, 0.0027, 0.0018, 0.0039, 0.0031, 0.0038, 0.0011, 0.0061,
          0.0016, 0.0050, 0.0063, 0.0020, 0.0027, 0.0041, 0.0042, 0.0024, 0.0046,
       

In [125]:
for i in range(6):
  decoder_input = torch.tensor([[output_indices[i]]], device=device)
  decoder_hidden = hidden
  output_size = output_lang.n_words
  embedded = embedding(decoder_input)
  attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
  attn_weights = F.softmax(attn_weights, dim = 1)
  attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
  input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
  input_to_gru = input_to_gru.unsqueeze(0)
  output, decoder_hidden = gru(input_to_gru, decoder_hidden)
  output = F.relu(output)
  output = F.softmax(output_word_layer(output[0]), dim = 1)
  top_value, top_index = output.data.topk(1)
  print(output_sentence.split(" ")[i], output_indices[i], output_lang.index2word[top_index.item()], top_index.item() )
  print(attn_weights)

i 2 excluding 624
tensor([[0.0013, 0.0038, 0.0024, 0.0021, 0.0026, 0.0025, 0.0041, 0.0021, 0.0027,
         0.0018, 0.0026, 0.0054, 0.0027, 0.0027, 0.0051, 0.0040, 0.0029, 0.0034,
         0.0042, 0.0019, 0.0029, 0.0048, 0.0041, 0.0025, 0.0043, 0.0046, 0.0030,
         0.0030, 0.0048, 0.0039, 0.0027, 0.0031, 0.0041, 0.0043, 0.0036, 0.0062,
         0.0038, 0.0056, 0.0038, 0.0034, 0.0036, 0.0051, 0.0024, 0.0021, 0.0017,
         0.0046, 0.0014, 0.0017, 0.0021, 0.0031, 0.0033, 0.0029, 0.0024, 0.0033,
         0.0034, 0.0018, 0.0015, 0.0049, 0.0031, 0.0020, 0.0046, 0.0033, 0.0043,
         0.0058, 0.0043, 0.0024, 0.0028, 0.0024, 0.0046, 0.0040, 0.0038, 0.0024,
         0.0029, 0.0022, 0.0022, 0.0028, 0.0016, 0.0035, 0.0038, 0.0048, 0.0023,
         0.0040, 0.0028, 0.0039, 0.0054, 0.0015, 0.0074, 0.0022, 0.0027, 0.0046,
         0.0045, 0.0050, 0.0020, 0.0032, 0.0044, 0.0030, 0.0021, 0.0035, 0.0033,
         0.0027, 0.0011, 0.0042, 0.0023, 0.0023, 0.0020, 0.0036, 0.0064, 0.0038,
         0

In [126]:
output_indices, output_sentence, input_sentence

([2,
  21,
  29,
  4,
  5,
  968,
  560,
  8,
  76,
  95,
  99,
  27,
  156,
  17,
  319,
  110,
  176,
  1260,
  217,
  332,
  1261,
  374,
  6,
  257,
  19,
  1],
 'i am a year old single male is it too late for me to find love would women be put off by my age ?',
 'i am a year old man . is it too late to find love . would my age put women off ?')

In [127]:
%matplotlib inline

In [128]:
# from __future__ import unicode_literals, print_function, division
# from io import open
# import unicodedata
# import string
# import re
# import random

# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [129]:
# device

device(type='cpu')

In [130]:
# !wget https://download.pytorch.org/tutorial/data.zip

# !unzip data.zip

--2021-11-25 16:04:25--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.226.52.90, 13.226.52.51, 13.226.52.128, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.52.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.1s    

2021-11-25 16:04:25 (21.3 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Itali

In [131]:
# SOS_token = 0
# EOS_token = 1


# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS"}
#         self.n_words = 2  # Count SOS and EOS

#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1

In [132]:
# # Turn a Unicode string to plain ASCII, thanks to
# # https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )

# # Lowercase, trim, and remove non-letter characters


# def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#     return s

In [133]:
# def readLangs(lang1, lang2, reverse=False):
#     print("Reading lines...")

#     # Read the file and split into lines
#     lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')

#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

#     # Reverse pairs, make Lang instances
#     if reverse:
#         pairs = [list(reversed(p)) for p in pairs]
#         input_lang = Lang(lang2)
#         output_lang = Lang(lang1)
#     else:
#         input_lang = Lang(lang1)
#         output_lang = Lang(lang2)

#     return input_lang, output_lang, pairs

In [158]:
MAX_LENGTH = 295

# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s ",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )


# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH and \
#         p[1].startswith(eng_prefixes)


# def filterPairs(pairs):
#     return [pair for pair in pairs if filterPair(pair)]

In [135]:
# def prepareData(lang1, lang2, reverse=False):
#     input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
#     print("Read %s sentence pairs" % len(pairs))
#     pairs = filterPairs(pairs)
#     print("Trimmed to %s sentence pairs" % len(pairs))
#     print("Counting words...")
#     for pair in pairs:
#         input_lang.addSentence(pair[0])
#         output_lang.addSentence(pair[1])
#     print("Counted words:")
#     print(input_lang.name, input_lang.n_words)
#     print(output_lang.name, output_lang.n_words)
#     return input_lang, output_lang, pairs


# input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
# print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['c est une fille mais elle est brave .', 'she s a girl but she s brave .']


In [159]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [160]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.gru(output, hidden)
        output = F.relu(output)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [161]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [162]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [163]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [164]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [165]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [166]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [167]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [146]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [147]:
evaluateRandomly(encoder1, attn_decoder1)

> il est un peu naif .
= he s a bit naive .
< he is a a a a a a a a

> tu es sournois .
= you re sneaky .
< you re . . . <EOS>

> je vais aller rejoindre les bras de morphee .
= i am going to sleep .
< i m . . . . . . . .

> nous sommes depourvues de talent .
= we re untalented .
< we re not . <EOS>

> nous sommes enfin seules .
= we re finally alone .
< we re . . . . . . . .

> c est une beaute .
= she s a cutie .
< he s a . <EOS>

> on m utilise .
= i m being used .
< i m . . <EOS>

> je suis honnete .
= i m honest .
< i m . <EOS>

> je suis satisfait .
= i m satisfied .
< i m . <EOS>

> je suis vraiment desole .
= i am truly sorry .
< i m . . <EOS>

